In [1]:
from pexpect import popen_spawn
import pexpect
import signal
import sys
import os
import pickle

class Metamath:
    def __init__(self, metamath_path):
        self.path = metamath_path
        self.process_dict = {}
        self.timeout = 3600            #some proofs took around 30 min to convert
        self.setup_signal_handler()

    def setup_signal_handler(self):
        # Define a handler for the SIGINT signal (Ctrl+C)
        def signal_handler(sig, frame):
            print("\nProcess terminated by user.")
            for id, process in self.process_dict.items():
                process.sendintr()  # Send Ctrl+C to all running processes
            sys.exit(0)

        # Register the signal handler
        signal.signal(signal.SIGINT, signal_handler)

    def initialize(self, id):
        self.process_dict[id] = popen_spawn.PopenSpawn(f"{self.path}")
        try:
            self.process_dict[id].expect('MM>', timeout=self.timeout)
        except pexpect.TIMEOUT:
            print(f"Timeout occurred while waiting for 'MM>' prompt during initialization.")
            self.terminate_process(self.process_dict[id])
            del self.process_dict[id]
            return None
        return self.process_dict[id].before.decode()


    def send(self, id, command):
        self.process_dict[id].sendline(command)
        try:
            self.process_dict[id].expect(r'MM>|MM-PA>|to scroll to end...', timeout=self.timeout)
        except pexpect.TIMEOUT:
            print(f"Timeout occurred while waiting for the response to the command '{command}'.")
            self.terminate_process(self.process_dict[id])
            del self.process_dict[id]
            return None
        return self.process_dict[id].before.decode()


    def terminate_process(self, process):
        # Send SIGINT signal to the process
        os.kill(process.pid, signal.SIGINT)

    def close(self, id):
        if id in self.process_dict:
            self.terminate_process(self.process_dict[id])
            del self.process_dict[id]



In [2]:
import re
from collections import defaultdict
import pandas as pd

df = pd.read_csv('tag.csv',na_values=['']).iloc[:]
df.head()

,tag,statement,antecedent_tag,hypothesis_s,hypothesis_v
0,wph,$f wff ph $.,NaN,NaN,NaN
1,wps,$f wff ps $.,NaN,NaN,NaN
2,wch,$f wff ch $.,NaN,NaN,NaN
3,wth,$f wff th $.,NaN,NaN,NaN
4,wta,$f wff ta $.,NaN,NaN,NaN


In [ ]:
except_dict={} 

In [ ]:
thrd=0
m=Metamath('./metamath')
print(m.initialize(thrd))
m.send(thrd,'READ "set.mm"')
print(m.send(thrd,'set height 1000000'))
print(m.send(thrd,'set width 1000000'))

count=0

for tag in df.iloc[count:].tag:
    try:
        m.send(thrd,f'save proof {tag} /normal')
    except Exception as e:
        except_dict[count]=tag
        print(f" error {e}:  {n} {tag}")
        thrd += 1
        print(m.initialize(thrd))
        print(m.send(thrd,f'read "set.mm"'))
        print(m.send(thrd,'set height 1000000'))
        print(m.send(thrd,'set width 100000')) 

    if count%999==0:
        print(count)
    count+=1
    
m.send(thrd,f'write source set_normal.mm')

for i in range(thrd+1):
    m.close(thrd)
